In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import GPT2Tokenizer, get_linear_schedule_with_warmup
import random
import numpy as np
# For reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

df = pd.read_csv("data/lyrics_midi_data.csv")
df.head()

,lyrics,midi_path
0,In sleep he sang to me\nin dreams he came\ntha...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
1,I have plans and schemes\nAnd I have hopes and...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
2,I get up and nothing gets me You got\nit tough...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
3,Man a hot like seven inches\nfrom the midday I...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
4,We come from the land of the ice and snow\nfro...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...


In [3]:
from torch.utils.data import random_split
from pathlib import Path
from miditok import TSD, TokenizerConfig

# Load MIDI Tokenizer
config = TokenizerConfig(
    use_velocities=False,
    use_chords=False,
    use_rests=False,
    use_tempos=False,
    use_time_signatures=False,
)

midi_tokenizer = TSD(config)
midi_tokenizer

# Load GPT-2 tokenizer
lyrics_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
lyrics_tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
lyrics_vocab_size = len(lyrics_tokenizer)

In [4]:
from data import randomize_midi_pitch, LyricsMidiDataset
from torch.utils.data import random_split

batch_size = 4

# Create dataset and dataloader
dataset = LyricsMidiDataset(df, lyrics_tokenizer, midi_tokenizer, max_length=512, root_dir='data', augment=True)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [5]:
from model import LyricsGenerator, generate_lyrics
from train import train, validate, save_checkpoint, load_checkpoint

# Instantiate model
model = LyricsGenerator(lyrics_vocab_size=len(lyrics_tokenizer), d_model=768, max_lyrics_length=512, max_midi_length=512)
model.to(device)
print("")

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
# Train the model
epochs = 6
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

train(model, train_dataloader, val_dataloader, optimizer, scheduler, epochs, device=device, lyrics_tokenizer=lyrics_tokenizer, save_every=3)

In [7]:
import os

base_dir = 'data/'
midi_path = os.path.join(base_dir, df["midi_path"][9001])
input_text = None
generated_lyrics = generate_lyrics(
    model=model,
    midi_path=midi_path,
    lyrics_tokenizer=lyrics_tokenizer,
    midi_tokenizer=midi_tokenizer,
    max_midi_length=256,
    max_lyrics_length=512,
    num_beams=5,
    input_text=input_text
)
print("Generated Lyrics:")
print(generated_lyrics)

Generated Lyrics:
I can't believe I'm reading this.

I can't believe I'm reading this.

I can't believe I'm reading this.

I can't believe I'm reading this.

I can't believe I'm reading this.

I can't believe I'm reading this.
